In [66]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
from sophius.db import database, Experiments, Models, ModelEpochs
import sqlite3
import pandas as pd

with database:
    database.create_tables([Experiments, Models, ModelEpochs])

with sqlite3.connect('../data/models.db') as conn:
    exp_df = pd.read_sql('SELECT * FROM experiments', conn)
    exp_df.id += 1
    exp_df.drop(columns='hash', inplace=True)
    
    models_df = pd.read_sql('SELECT * FROM models', conn)
    models_df.id += 1
    models_df.exp_id = models_df.exp_id.astype(int) + 1
    
    numeric_cols = models_df.columns.tolist()
    numeric_cols.remove('hash')
    for col in numeric_cols:
        models_df[col] = pd.to_numeric(models_df[col], errors='coerce')        
    for col in ['val_acc', 'train_acc']:
        models_df[col] = models_df[col].round(4)
    models_df.time = models_df.time.round(1)
    
    epochs_df = pd.read_sql('SELECT * FROM model_epochs', conn)
    for col in epochs_df.columns:
        epochs_df[col] = pd.to_numeric(epochs_df[col], errors='coerce')
    epochs_df.exp_id += 1
    epochs_df.model_id += 1    
    epochs_df.epoch = epochs_df.epoch.astype(int)
    for col in ['val_acc', 'train_acc']:
        epochs_df[col] = epochs_df[col].round(4)
    epochs_df.loss = epochs_df.loss.round(2)
    epochs_df.time = epochs_df.time.round(1)
    
epochs_df

,epoch,loss,train_acc,val_acc,time,model_id,exp_id
0,0,240.27,0.5451,0.5290,0.4,1,1
1,1,190.27,0.6085,0.5800,0.7,1,1
2,2,171.38,0.6453,0.6024,1.0,1,1
3,3,159.32,0.6731,0.6152,1.3,1,1
4,4,150.31,0.6932,0.6242,1.6,1,1
...,...,...,...,...,...,...,...
426545,45,1.18,0.9974,0.7528,50.6,8531,1
426546,46,1.00,0.9981,0.7551,51.6,8531,1
426547,47,0.87,0.9983,0.7556,52.7,8531,1
426548,48,0.77,0.9982,0.7558,53.7,8531,1


In [68]:
exp_params = exp_df.to_dict(orient='records')
exp_params

[{'id': 1,
  'val_size': 10000,
  'batch_size': 256,
  'num_epoch': 50,
  'random_seed': 42,
  'optimizer': 'AdamW',
  'opt_params': "{'lr': 0.001}",
  'scheduler': 'ExponentialLR',
  'sch_params': "{'gamma': 0.95}",
  'in_shape': '(3, 32, 32)',
  'out_shape': '10'}]

In [69]:
from tqdm import tqdm

print('Insert experiments')    
for exp_params in exp_df.to_dict(orient='records'):    
    Experiments.insert(**exp_params).on_conflict_ignore().execute()
    
print('Insert models')    
Models.insert_many(models_df.to_dict(orient='records')).on_conflict_ignore().execute()    
print('Insert epochs')
for i in tqdm(models_df.id.unique()):
    mask = epochs_df.model_id == i
    ModelEpochs.insert_many(epochs_df[mask].to_dict(orient='records')).on_conflict_ignore().execute()

Insert experiments
Insert models
Insert epochs


100%|██████████| 8531/8531 [00:49<00:00, 172.86it/s]
